<a href="https://colab.research.google.com/github/AmanKabra/OB-Semantic-Analysis/blob/master/OB_Semantic_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -qq twint
!pip install -qq whatthelang
!pip install nest_asyncio
!pip install tweet-preprocessor
!pip install --upgrade gensim
!pip install transformers
!pip install contractions

     |████████████████████████████████| 1.2MB 5.8MB/s 
     |████████████████████████████████| 245kB 16.0MB/s 
     |████████████████████████████████| 225kB 16.6MB/s 
     |████████████████████████████████| 266kB 19.1MB/s 
     |████████████████████████████████| 153kB 19.2MB/s 
     |████████████████████████████████| 235kB 18.7MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 788kB 3.5MB/s 
     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 245kB 17.0MB/s 
     |████████████████████████████████| 24.2MB 127kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 15.6MB/s 
     |████████████████████████████████| 890kB 22.8MB/s 
     |████████████████████████████████| 3.0MB 32.2MB/s 
  Created wheel for sacremoses: filename=sacrem

In [ ]:
# Import Library
import twint
import pandas as pd
import numpy as np
import nest_asyncio
from multiprocessing import Process
nest_asyncio.apply()
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from collections import Counter
import json
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor.api as p
from preprocessor.api import clean, tokenize, parse
import seaborn as sns   # for charts
sns.set_style("whitegrid");   # chart background style
plt.rcParams['figure.dpi'] = 360   # for high res chart output
import spacy   # for tokenising text
from spacy.lang.en import English  # for tokenising text
nlp = English()   # for tokenising text
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import torch
import tensorflow as tf
from transformers import BertTokenizer
import contractions

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words("english"))
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Scrape: The telegraph ,The times, The Economist, The Guardian, BBC news UK, The Independent, and The Financial Times.

Handles: ['Telegraph', 'thetimes','TheEconomist','guardiannews','BBCNews','Independent','FinancialTimes']


# Preprocessing and EDA

In [ ]:
Telegraph = pd.DataFrame()
thetimes = pd.DataFrame()
TheEconomist = pd.DataFrame()
guardiannews = pd.DataFrame()
BBCNews = pd.DataFrame()
Independent = pd.DataFrame()
FinancialTimes = pd.DataFrame()

In [ ]:
c = twint.Config()
c.Store_object = True
c.Pandas = True
c.Username = "FinancialTimes"
c.Since = "2020-01-01"
c.Until = "2020-08-01"
twint.run.Search(c)


In [ ]:
 FinancialTimes = FinancialTimes.append(twint.storage.panda.Tweets_df, ignore_index = True)
 FinancialTimes.to_csv('FinancialTimes.csv')

In [ ]:
 Independent = Independent.append(twint.storage.panda.Tweets_df, ignore_index = True)
 Independent.to_csv('Independent.csv')

In [ ]:
 BBCNews = BBCNews.append(twint.storage.panda.Tweets_df, ignore_index = True)
 BBCNews.to_csv('BBCNews.csv')

In [ ]:
 guardiannews = guardiannews.append(twint.storage.panda.Tweets_df, ignore_index = True)
 guardiannews.to_csv('GuardianNews.csv')

In [ ]:
 TheEconomist = TheEconomist.append(twint.storage.panda.Tweets_df, ignore_index = True)
 TheEconomist.to_csv('TheEconomist.csv')

In [ ]:
 thetimes = thetimes.append(twint.storage.panda.Tweets_df, ignore_index = True)
 thetimes.to_csv('TheTimes.csv')

In [ ]:
 Telegraph = Telegraph.append(twint.storage.panda.Tweets_df, ignore_index = True)
 Telegraph.to_csv('Telegraph.csv')

Scraping done.

We now proceed toward NLP.

Step 1: Data cleaning and preprocessing

In [ ]:
BBCNews = pd.read_csv("BBCNews.csv")
FinancialTimes = pd.read_csv("FinancialTimes.csv")
Independent = pd.read_csv("Independent.csv")
Telegraph = pd.read_csv("Telegraph.csv")
TheEconomist = pd.read_csv("TheEconomist.csv")
guardiannews = pd.read_csv("guardiannews.csv")
thetimes = pd.read_csv("thetimes.csv")

ParserError: ignored

In [ ]:
fulldf = pd.DataFrame()

In [ ]:
fulldf = BBCNews
fulldf = fulldf.append(FinancialTimes, ignore_index = True)
fulldf = fulldf.append(Independent, ignore_index = True)
fulldf = fulldf.append(Telegraph, ignore_index = True)
fulldf = fulldf.append(TheEconomist, ignore_index = True)
fulldf = fulldf.append(guardiannews, ignore_index = True)
fulldf = fulldf.append(thetimes, ignore_index = True)

NameError: ignored

In [ ]:
df = fulldf[['tweet', 'hashtags','username','nlikes','nreplies','nretweets']]

In [ ]:
print(f"Total number of tweets: {len(df)}")

In [ ]:
# let's find out how many tweets contain a hashtag
tweets_with_hashtags = df.loc[df["tweet"].str.contains("#")]

# view the number of tweets that contain a hashtag
print(f"Number of tweets containing hashtags: {len(tweets_with_hashtags)}")

# view the tweets that contain a hashtag
tweets_with_hashtags

In [ ]:
# how many tweets contain a URL i.e. "http"?
tweets_with_URLs = df.loc[df["tweet"].str.contains("http")]

# view the number of tweets that contain a URL
print(f"Number of tweets containing URLs: {len(tweets_with_URLs)}")

# view the tweets that contain a URL
tweets_with_URLs

In [ ]:
# create a single string containing all the tweets, as this will be needed to be able to create a wordcloud
tweet_string = " ".join(tweet for tweet in df["tweet"])

# replace all the mentions (e.g. @Anurag_Gupta) from the tweets with '@USER'
tweet_string = re.sub(r'@\w+','@USER ', tweet_string)

# replace all the URLs with '_URL_'
tweet_string = re.sub(r'http\S+','_URL_ ', tweet_string)

# convert the text to lower case so, for example, instead of having "Be" and "be" included
# as 2 separate words, we'd only have "be"
tweet_string = tweet_string.lower()

# remove extra white spaces so there is only one space between words
tweet_string = re.sub(r'\s+',' ', tweet_string)

# view the first 200 elements of the string to check this worked as expected
tweet_string[0:200]

In [ ]:
# create the wordcloud
tweet_wordcloud = WordCloud(background_color="white", 
                              max_words=100, 
                             ).generate(tweet_string)

# view the wordcloud
plt.imshow(tweet_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

Trying another alternative (by putting empty space in place of URLs and Tags)

In [ ]:
# create a single string containing all the tweets, as this will be needed to be able to create a wordcloud
tweet_string = " ".join(tweet for tweet in df["tweet"])

# replace all the mentions (e.g. @Anurag_Gupta) from the tweets with '@USER'
tweet_string = re.sub(r'@\w+',' ', tweet_string)

# replace all the URLs with '_URL_'
tweet_string = re.sub(r'http\S+',' ', tweet_string)

# convert the text to lower case so, for example, instead of having "Be" and "be" included
# as 2 separate words, we'd only have "be"
tweet_string = tweet_string.lower()

# remove extra white spaces so there is only one space between words
tweet_string = re.sub(r'\s+',' ', tweet_string)

# view the first 200 elements of the string to check this worked as expected
tweet_string[0:200]

In [ ]:
# create the wordcloud
tweet_wordcloud = WordCloud(background_color="white", 
                              max_words=100, 
                             ).generate(tweet_string)

# view the wordcloud
plt.imshow(tweet_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

Running NLP

In [ ]:
# create a spacy document by pointing spacy to the tweet string
tweet_doc = nlp(tweet_string[0:1000000])

# get all tokens that aren't punctuation
tweet_words = [token.text for token in tweet_doc if token.is_punct != True]

# get the frequency of each word (token) in the tweet string
tweet_word_freq = Counter(tweet_words)

# get the 5 most frequent words
five_most_common_words = tweet_word_freq.most_common(5)

# view the 5 most common words
five_most_common_words

In [ ]:
# get all tokens that aren't punctuation and aren't stopwords
tweet_words = [token.text for token in tweet_doc if token.is_punct != True and \
token.is_stop != True]

# get the frequency of each word (token) in the tweet string
tweet_word_freq = Counter(tweet_words)

# re-create the Pandas dataframe containing the 
# tokens (words) and their frequencies
freq_df = pd.DataFrame.from_dict(tweet_word_freq, orient='index').reset_index()

# rename the columns to "word" and "freq"
freq_df.columns=["word", "freq"]

# display a bar chart showing the top 25 words and their
# frequencies (which will exclude the stopwords this time)
fig, ax = plt.subplots(figsize=(12,6))
sns.barplot(data=freq_df.sort_values(by="freq", ascending=False).head(25), 
            y="word", 
            x="freq", 
            color='#7bbcd5')
plt.ylabel("Word")
plt.xlabel("Frequency")
plt.title("Top 25 Most Frequent Words (Excluding Stopwords)")
plt.xticks([0,1,2,3])
sns.despine();

Word cloud without stop words

In [ ]:
# get all tokens that aren't punctuation and aren't stopwords
tweet_words = [token.text for token in tweet_doc if token.is_punct != True and \
token.is_stop != True]

# get the frequency of each word (token) in the tweet string
tweet_word_freq = Counter(tweet_words)

# re-create the Pandas dataframe containing the 
# tokens (words) and their frequencies
freq_df = pd.DataFrame.from_dict(tweet_word_freq, orient='index').reset_index()

# rename the columns to "word" and "freq"
freq_df.columns=["word", "freq"]

freq_df

In [ ]:
listToStr = ' '.join([str(elem) for elem in tweet_words]) 

# create the wordcloud
tweet_wordcloud = WordCloud(background_color="white", 
                              max_words=100, 
                             ).generate(listToStr)

# view the wordcloud
plt.imshow(tweet_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
def remove_content(text):
    text = re.sub(r"http\S+", "", text) #remove urls
    text=re.sub(r'\S+\.com\S+','',text) #remove urls
    text=re.sub(r'\@\w+','',text) #remove mentions
    text =re.sub(r'\#\w+','',text) #remove hashtags
    return text
def process_text(text, stem=False): #clean text
    text=remove_content(text)
    text = re.sub('[^A-Za-z]', ' ', text.lower()) #remove non-alphabets
    tokenized_text = word_tokenize(text) #tokenize
    clean_text = [
         word for word in tokenized_text
         if word not in stop_words
    ]
    if stem:
        clean_text=[wordnet_lemmatizer.lemmatize(word) for word in clean_text]
    return ' '.join(clean_text)

In [ ]:
df['cleaned_tweets']=df['tweet'].apply(lambda x: process_text(x, True))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


pandas.core.series.Series

In [ ]:
temp=' '.join(df['cleaned_tweets'].tolist())
wordcloud = WordCloud(width = 800, height = 500, 
                background_color ='white', 
                min_font_size = 10).generate(temp)
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0) 
plt.show()

In [ ]:
from textblob import TextBlob
df['sentiment']=df['tweet'].apply(lambda x:TextBlob(x).sentiment[0])
df['subject']=df['tweet'].apply(lambda x: TextBlob(x).sentiment[1])
df['polarity']=df['sentiment'].apply(lambda x: 'pos' if x>=0 else 'neg')

In [ ]:
import collections
c = collections.Counter(df['polarity'])
c = sorted(c.items())
polarity = [i[0] for i in c]
freq = [i[1] for i in c]

f, ax = plt.subplots()

plt.bar(polarity, freq)
plt.title("Polarity of tweets")
plt.xlabel("Polarity")
plt.ylabel("Frequency")
plt.show()

Word2Vec

In [ ]:
#coviddf = df[df['tweet'].str.contains("COVID|Corona|Coronavirus|SARS-CoV-2|COVID-19|Ncov|Vaccination|Vaccine|Quarantine|Pandemic|Social Distancing|Distancing|NHS|Doctor|Outbreak|PPE|Sanitiser|Wuhan|Death|Lockdown|Bat|Virus|Mask|Respirator|Lung|China|Infect|Isolation|Ventilator|Test kits|Recession|Testkit|Disease|Transmission|Transmit|AstraZeneca|GeneXpert|RT-PCR|RTPCR|Oxford|Community|Spread|Herd|Comorbid|Flatten|Curve|Immunity|Antibodies|Antibody|Mortality|Treatment|Preventive|Prevention|Symptom|Throat|Cure|Die|Sick|World Health|Fomite|Tracing|Trace|Patient|Contagious|Gather|Restrict|Novel|Countrywide|Incubation|Drug|Hydroxychloroquine|Essential|Crisis|Containment|Contactless|Epidemiology|Thermometer|Surveillance|Immunosuppress|Incubation|Index case|Index patient|National emergency|Intensivist|Phlebotomist|Patient zero|Remdesivir|PUI|Respirator|Shelter-in-place|Spanish flu|Super-spreader|Pneumonia|Kung flu|Faceshield|Restaurants closed|Bars closed")]

In [ ]:
urlarray = [] 
for line in df["tweet"]:
  url = re.findall('(?P<url>https?://[^\s]+)', line)
  urlarray.append(url)

df["URL1"] = urlarray

In [ ]:
urlarray = [] 
for line in df["tweet"]:
  url = re.findall('((?:https?://)?(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)', line)
  urlarray.append(url)
df["URL2"] = urlarray

In [ ]:
df['URL1'].equals(df['URL2'])

In [ ]:
len(df[df['URL1'] != df['URL2']])

In [ ]:
data = []
for i in sent_tokenize(df['cleaned_tweets']): 
    temp = []      
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
    data.append(temp)

In [ ]:
tt = TweetTokenizer()
b=[]
for t in df["tweet"]:
  a = tt.tokenize(t)
  b.append(a)

In [ ]:
# train model
model = Word2Vec(b, min_count=1)
# summarize vocabulary
words = list(model.wv.vocab)
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')

In [ ]:
model

# BERT

In [ ]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# If there's a GPU available
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
BBCNews = pd.read_csv("BBCNews.csv")
FinancialTimes = pd.read_csv("FinancialTimes.csv")
Independent = pd.read_csv("Independent.csv")
Telegraph = pd.read_csv("Telegraph.csv")
TheEconomist = pd.read_csv("TheEconomist.csv")
guardiannews = pd.read_csv("guardiannews.csv")
thetimes = pd.read_csv("thetimes.csv")

fulldf = pd.DataFrame()

fulldf = BBCNews
fulldf = fulldf.append(FinancialTimes, ignore_index = True)
fulldf = fulldf.append(Independent, ignore_index = True)
fulldf = fulldf.append(Telegraph, ignore_index = True)
fulldf = fulldf.append(TheEconomist, ignore_index = True)
fulldf = fulldf.append(guardiannews, ignore_index = True)
fulldf = fulldf.append(thetimes, ignore_index = True)

df = fulldf[['tweet', 'hashtags','date','username','nlikes','nreplies','nretweets']]

cdf = df[df['tweet'].str.contains("COVID|Corona|Coronavirus|SARS-CoV-2|COVID-19|Ncov|Vaccination|Vaccine|Quarantine|Pandemic|Social Distancing|Distancing|NHS|Doctor|Outbreak|PPE|Sanitiser|Wuhan|Death|Lockdown|Bat|Virus|Mask|Respirator|Lung|China|Infect|Isolation|Ventilator|Test kits|Recession|Testkit|Disease|Transmission|Transmit|AstraZeneca|GeneXpert|RT-PCR|RTPCR|Oxford|Community|Spread|Herd|Comorbid|Flatten|Curve|Immunity|Antibodies|Antibody|Mortality|Treatment|Preventive|Prevention|Symptom|Throat|Cure|Die|Sick|World Health|Fomite|Tracing|Trace|Patient|Contagious|Gather|Restrict|Novel|Countrywide|Incubation|Drug|Hydroxychloroquine|Essential|Crisis|Containment|Contactless|Epidemiology|Thermometer|Surveillance|Immunosuppress|Incubation|Index case|Index patient|National emergency|Intensivist|Phlebotomist|Patient zero|Remdesivir|PUI|Respirator|Shelter-in-place|Spanish flu|Super-spreader|Pneumonia|Kung flu|Faceshield|Restaurants closed|Bars closed")]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#creating URL column

urlarray = [] 
for line in cdf["tweet"]:
  url = re.findall('((?:https?://)?(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)', line)
  urlarray.append(url)
cdf["URL"] = urlarray

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
#creating fuction to remove unwanted parts from COVID tweets

def remove_content(text):
  text = re.sub(r"http\S+", "", text) #remove urls
  text=re.sub(r'\S+\.com\S+','',text) #remove urls
  text=re.sub(r'\@\w+','',text) #remove mentions
  text =re.sub(r'\#\w+','',text) #remove hashtags
  text = contractions.fix(text) #replace contractions with full forms (eg. don't becomes do not)
  text = re.sub('[^A-Za-z]', ' ', text.lower()) #remove non-alphabets
  text = text.lower() #converts to lower case
  text = re.sub(r'\s+',' ', text) #remove extra white space
  return text

#calling the function
cdf['cleaned_tweets']=cdf['tweet'].apply(lambda x: remove_content(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
 cdf.to_csv('cdf.csv')

In [ ]:
# Get the lists of sentences and their labels.
sentences = cdf.cleaned_tweets.values

In [ ]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

In [ ]:
max_len = 0

# For every sentence...
for sent in sentences:
  # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
  input_ids = tokenizer.encode(sent, add_special_tokens=True)
  # Update the maximum sentence length.
  max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)
print('This is not as per character length, but as per "token" length according to BERT token creations')

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])